<a href="https://colab.research.google.com/github/hibadash/-Breast-Ultrasound-Classification-Using-Xception-CNN-BUSI-Dataset/blob/main/notebooks/02_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook 02 — Entraînement du modèle

In [ ]:
# test save file 2

In [1]:
# Cloner le repo GitHub dans un dossier local "breast_project" (Vérification chaque fois mais c'est rapide)
import os
# Vérifie si le dossier "breast_project" existe, sinon clone dedans
if not os.path.exists("breast_project"):
    !git clone https://github.com/hibadash/-Breast-Ultrasound-Classification-Using-Xception-CNN-BUSI-Dataset.git breast_project
# Aller dans le dossier cloné
%cd breast_project
# Vérifier le contenu du projet
!ls

Cloning into 'breast_project'...
remote: Enumerating objects: 1636, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1636 (delta 24), reused 14 (delta 1), pack-reused 1584 (from 2)
Receiving objects: 100% (1636/1636), 194.53 MiB | 19.70 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Updating files: 100% (1589/1589), done.
/content/breast_project
app  data  notebooks  README.md  requirements.txt  results  src
